In [ ]:
# !pip download -d packages transformers
!pip install -q -U transformers --no-index -f /kaggle/input/metadino-v3-convnext/pytorch/default/3/packages
# !pip install -q -U transformers xgboost opencv-python 'numpy<2.0' 'pandas>=2.2'

In [ ]:
# import huggingface_hub
# huggingface_hub.login()

In [ ]:
import os
import glob
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm

import torch
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Explore Data

In [ ]:
PATH_DATA = '/kaggle/input/csiro-biomass'
PATH_TRAIN_CSV = os.path.join(PATH_DATA, 'train.csv')
PATH_TRAIN_IMG = os.path.join(PATH_DATA, 'train')
PATH_TEST_IMG = os.path.join(PATH_DATA, 'test')

df = pd.read_csv(PATH_TRAIN_CSV)
print(f"Dataset size: {df.shape}")
display(df.head())

In [ ]:
TARGET_COLS = [c for c in df.columns if c not in ['image_id', 'Image']]
print(f"Target columns: {TARGET_COLS}")
print(f"Number of targets: {len(TARGET_COLS)}")

## Plot target distribution

In [ ]:
# Exclude non-numeric or identifier columns from histogram plotting
cols_to_plot = [col for col in TARGET_COLS if col not in ['sample_id', 'image_path', 'State', 'target_name']]

for col in cols_to_plot:
    plt.figure(figsize=(8, 3)) # Create a new figure for each histogram
    plt.hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.title(f'{col} Distribution', fontsize=14, fontweight='bold')
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45, ha="right") # Rotate x-axis labels
    plt.tight_layout() # Adjust layout to prevent overlap
    plt.show()

In [ ]:
cols_to_plot = ['State', 'target_name']
n_rows, n_cols = 1, len(cols_to_plot)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))

# Ensure axes is an array even for a single subplot
axes = axes.flatten()

for ax, col in zip(axes, cols_to_plot):
    counts = df[col].value_counts()
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=140)
    ax.set_title(f'Distribution of {col}', fontsize=14, fontweight='bold')
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

grouped_train_data = df.groupby('target_name')

plt.figure(figsize=(12, 8))
for target_name, group_data in grouped_train_data:
    sns.histplot(data=group_data, x='target', kde=True, label=target_name)

plt.title('Distribution of Target for Each Target Name Class')
plt.xlabel('Target')
plt.ylabel('Frequency')
plt.legend(title='Target Name')
plt.grid(True) # Added grid here
plt.show()

## Show sample images

In [ ]:
def show_images(df_sample, n=12, path_img=PATH_DATA):
    """Displays a linear sampling of images sorted by target value."""

    # Sort the DataFrame by the 'target' column
    df_sorted = df_sample.sort_values(by='target').reset_index(drop=True)

    # Perform linear sampling
    indices_to_show = np.linspace(0, len(df_sorted) - 1, n, dtype=int)
    df_to_show = df_sorted.iloc[indices_to_show]

    # Determine the number of rows and columns for subplots
    n_cols = 3  # You can adjust this number
    n_rows = (n + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
    axes = axes.flatten()

    # Remove unused subplots if any
    for i in range(n, len(axes)):
        fig.delaxes(axes[i])

    for i, (idx, row) in enumerate(df_to_show.iterrows()):
        # Use image_path directly (includes train/ID....jpg)
        img_path = os.path.join(path_img, row['image_path'])

        if os.path.exists(img_path):
            img = Image.open(img_path).convert('RGB')
            axes[i].imshow(img)
            # Include the target value in the title
            title = f"ID: {row['sample_id']}\nTarget: {row['target']:.2f}"
            axes[i].set_title(title, fontsize=10)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage: Show 12 images linearly sampled based on target value
show_images(df, n=12)

# Feature extraction

In [ ]:
from transformers import pipeline

# Define the feature extraction pipeline
feature_extractor = pipeline(
    # model="facebook/dinov3-convnext-tiny-pretrain-lvd1689m",
    model="/kaggle/input/metadino-v3-convnext/pytorch/default/3/dinov3-convnext-tiny-pretrain-lvd1689m",
    task="image-feature-extraction",
    device=0 if torch.cuda.is_available() else -1 # Use GPU if available
)

def extract_image_features_pipeline(image_path, feature_extractor, path_data):
    """
    Splits an image into two 1000x1000 parts and extracts features from each using a pipeline.
    """
    full_image_path = os.path.join(path_data, image_path)
    img = Image.open(full_image_path)#.convert('RGB')

    # Split the image into two 1000x1000 parts
    width, height = img.size
    img1 = img.crop((0, 0, width // 2, height))
    img2 = img.crop((width // 2, 0, width, height))

    # The pipeline expects PIL Images or paths, use 'inputs'
    extracted_features = feature_extractor(inputs=[img1, img2], pool=True)
    # extracted_features = feature_extractor(inputs=[img], pool=True)
    # print(f"extracted_features: {np.array(extracted_features).shape}")

    # Concatenate features from both parts
    combined_features = list(np.array(extracted_features).flatten())
    # print(f"combined_features: {len(combined_features)}")

    return combined_features

print("Image feature extraction function defined using pipeline.")

In [ ]:
# Extract features for all images
image_features = {}
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    image_path = row['image_path']
    # Use the pipeline function
    features = extract_image_features_pipeline(image_path, feature_extractor, PATH_DATA)
    # Use the sample_id as the key for the features
    image_features[row['sample_id']] = features

print("Image feature extraction complete.")

In [ ]:
# Convert the image features dictionary to a DataFrame
image_features_df = pd.DataFrame.from_dict(image_features, orient='index')
image_features_df.index.name = 'sample_id'
image_features_df.columns = [f'img_feature_{i}' for i in range(image_features_df.shape[1])]

# One-hot encode the 'target_name' column
df_one_hot = pd.get_dummies(df['target_name'], prefix='target_name').astype(int) # Convert to int

# Merge the image features and one-hot encoded features with the original DataFrame
df_combined = df.merge(image_features_df, on='sample_id', how='left')
df_combined = df_combined.merge(df_one_hot, left_index=True, right_index=True)

print("Image features and one-hot encoded target names combined with the original DataFrame.")
display(df_combined.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Flatten all image features into a single Series
image_features_cols = [col for col in df_combined.columns if col.startswith("img_feature_")]
all_features_flat = image_features_df[image_features_cols].values.flatten()
all_features_series = pd.Series(all_features_flat)

# Plot a single histogram for all features
plt.figure(figsize=(10, 6))
sns.histplot(all_features_series, kde=True, bins=100) # Adjust bins as needed
plt.title('Distribution of All Extracted Image Feature Values')
plt.xlabel('Feature Value')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.show()

# Train Regressor

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Separate image features and other features
image_feature_cols = [col for col in df_combined.columns if col.startswith('img_feature_')]
other_features_cols = [col for col in df_combined.columns if col.startswith('target_name_')]

X_image_features = df_combined[image_feature_cols]
X_other_features = df_combined[other_features_cols]

# Standardize the image features before applying PCA
scaler = StandardScaler()
X_image_scaled = scaler.fit_transform(X_image_features)

# Apply PCA to reduce dimensionality
# You can adjust the number of components (n_components)
pca = PCA(n_components=75) # Example: Reduce components by 20x
X_image_pca = pca.fit_transform(X_image_scaled)

# Convert the PCA reduced features to a DataFrame
X_image_pca_df = pd.DataFrame(
    X_image_pca, index=df_combined.index,
    columns=[f'pca_img_feature_{i}' for i in range(pca.n_components)])

# Combine the PCA reduced image features with the other features
X_combined_pca = pd.concat([X_other_features, X_image_pca_df], axis=1)

print(f"Original number of image features: {X_image_features.shape[1]}")
print(f"Reduced number of image features after PCA: {X_image_pca_df.shape[1]}")
print(f"Total number of features for regression after PCA: {X_combined_pca.shape[1]}")

# Display the first few rows of the combined feature DataFrame after PCA
display(X_combined_pca.head())

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Define features (X) and target (y) using the PCA-reduced image features
# Exclude identifier columns and the original target column
# Use X_combined_pca which contains the one-hot encoded target_name and PCA image features
features = X_combined_pca.columns.tolist() # Use all columns from X_combined_pca
print(f"training features: {features}")
target = 'target'

X = X_combined_pca
y = df_combined[target] # Target remains the same

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost Regressor
# The features include binary (from one-hot encoding) and float types.
# XGBoost can handle this mix of feature types effectively.

# Define a distribution for hyperparameters to sample from
param_dist = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    #'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.001, 0.01, 0.1, 0.5, 1, 5, 10],
    'reg_lambda': [1, 1.5, 2, 5, 10]
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Use RandomizedSearchCV to find the best parameters
# n_iter controls the number of parameter settings that are sampled
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=500,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)

random_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_params = random_search.best_params_
xgb_model = random_search.best_estimator_

print("Randomized search complete.")
print(f"Best parameters found: {best_params}")
print("XGBoost model training complete with best parameters.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances from the best model found by GridSearchCV
# If you didn't use GridSearchCV, use xgb_model.feature_importances_
feature_importances = xgb_model.feature_importances_

# Create a pandas Series for easier handling and sorting
feature_importances_series = pd.Series(feature_importances, index=X_train.columns)

# Sort features by importance
sorted_feature_importances = feature_importances_series.sort_values(ascending=False)

# Plot the top N feature importances (e.g., top 20)
n_top_features = 20
plt.figure(figsize=(10, 8))
sns.barplot(x=sorted_feature_importances.head(n_top_features).values,
            y=sorted_feature_importances.head(n_top_features).index,
            palette='viridis')
plt.title(f'Top {n_top_features} Feature Importances (XGBoost)', fontsize=14)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.show()

# You can also display the sorted feature importances
print("Sorted Feature Importances:")
display(sorted_feature_importances)

In [ ]:
# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

print("Predictions on test data complete.")

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE) on the test set: {rmse}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE) on the test set: {mae}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared on the test set: {r2}")

# Test predictions

In [ ]:
# 1. Load test data
PATH_TEST_CSV = os.path.join(PATH_DATA, 'test.csv')
df_test = pd.read_csv(PATH_TEST_CSV)
print(f"Test Dataset size: {df_test.shape}")
display(df_test.head())

In [ ]:
# 2. Extract test image features
test_image_features = {}
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    image_path = row['image_path']
    # Use the pipeline function
    features = extract_image_features_pipeline(image_path, feature_extractor, PATH_DATA)
    # Use the sample_id as the key for the features
    test_image_features[row['sample_id']] = features

print("Test Image feature extraction complete.")

In [ ]:
# 3. Combine test features
test_image_features_df = pd.DataFrame.from_dict(test_image_features, orient='index')
test_image_features_df.index.name = 'sample_id'
test_image_features_df.columns = [f'img_feature_{i}' for i in range(test_image_features_df.shape[1])]

# One-hot encode the 'target_name' column for the test set
test_df_one_hot = pd.get_dummies(df_test['target_name'], prefix='target_name').astype(int)

# Merge the test image features and one-hot encoded features with the original test DataFrame
df_test_combined = df_test.merge(test_image_features_df, on='sample_id', how='left')
df_test_combined = df_test_combined.merge(test_df_one_hot, left_index=True, right_index=True)

print("Test image features and one-hot encoded target names combined.")
display(df_test_combined.head())

In [ ]:
# 4. Scale and PCA transform test image features
# Separate image features and other features for the test set
test_image_feature_cols = [col for col in df_test_combined.columns if col.startswith('img_feature_')]
test_other_features_cols = [col for col in df_test_combined.columns if col.startswith('target_name_')]

X_test_image_features = df_test_combined[test_image_feature_cols]
X_test_other_features = df_test_combined[test_other_features_cols]

# Apply the *trained* scaler and PCA to the test image features
X_test_image_scaled = scaler.transform(X_test_image_features) # Use the fitted scaler
X_test_image_pca = pca.transform(X_test_image_scaled)       # Use the fitted pca

# Convert the PCA reduced features to a DataFrame
X_test_image_pca_df = pd.DataFrame(
    X_test_image_pca, index=df_test_combined.index,
    columns=[f'pca_img_feature_{i}' for i in range(pca.n_components)])

print(f"Scaled and PCA-transformed test image features shape: {X_test_image_pca_df.shape}")

# Display the first few rows of the PCA reduced test image features
display(X_test_image_pca_df.head())

In [ ]:
# 5. Prepare test data for inference
# Combine the PCA reduced image features with the other features (one-hot encoded target_name)
X_test_final = pd.concat([X_test_other_features, X_test_image_pca_df], axis=1)

print(f"Final test feature DataFrame shape: {X_test_final.shape}")

# Display the first few rows of the final test feature DataFrame
display(X_test_final.head())

In [ ]:
# 6. Make predictions
test_predictions = xgb_model.predict(X_test_final)

print("Predictions on test data complete.")
display(test_predictions)

# Prepare submission

In [ ]:
# 7. Generate submission file
submission_df = pd.DataFrame({'sample_id': df_test['sample_id'], 'target': test_predictions})

# Ensure predictions are non-negative if the target represents a physical quantity
submission_df['target'] = submission_df['target'].clip(lower=0)

submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission file generated at: {submission_path}")
display(submission_df.head())

In [ ]:
!head submission.csv